# Visualising different normalisation

In [1]:
import claud_ex
import utils

In [3]:
sample = './media/69241.mp4'
frames = utils.load_rgb_frames_from_video(sample, 0, 0, all=True)
print(frames.shape)

torch.Size([64, 3, 256, 256])


In [4]:
import torch
import torchvision.transforms as transforms
import torch.nn.functional as F
import random
import numpy as np

In [5]:
base_mean = [0.43216, 0.394666, 0.37645]
base_std = [0.22803, 0.22145, 0.216989]
base_transform = transforms.Compose([
  transforms.Lambda(lambda x: correct_num_frames(x, 16)),
  transforms.Lambda(lambda x: x.float() / 255.0),
  transforms.Lambda(lambda x: F.interpolate(x, size=(112, 112), 
                    mode='bilinear', align_corners=False)),
])
    

### Visualise base

## Random normalisation

In [6]:
augment_prob=0.5
rand_norm_aug = claud_ex.RandomNormalizationAugmentation(
  base_mean=base_mean,
  base_std=base_std,
  mean_var=0.05,
  std_var=0.03,
  prob=augment_prob
)

In [8]:
permute_fin = lambda x: x.permute(1,0,2,3)

based_rand_norm_aug = transforms.Compose([
  base_transform,
  transforms.Lambda(rand_norm_aug),
  transforms.Lambda(permute_fin)
])

## Adaptive normalisation

In [20]:

base_mean = base_mean.clone().detach().float()
base_std = base_std.clone().detach().float()
print(f'base mean: {base_mean}')
print(f'base std: {base_std}')


video_mean = frames.float().mean(dim=[0,2,3])
video_std = frames.float().std(dim=[0,2,3])

video_mean = video_mean.clone().detach().float()
video_std = video_std.clone().detach().float()
print(f'Video mean: {video_mean}')
print(f'Video std: {video_std}')
adm = (1-0.3) * base_mean + 0.3 * video_mean
ads = (1 - 0.3) * base_std + 0.3 * video_std
print(f'adaptive mean: {adm}')
print(f'adaptive std: {ads}')


base mean: tensor([0.4322, 0.3947, 0.3765])
base std: tensor([0.2280, 0.2215, 0.2170])
Video mean: tensor([65.9386, 71.3979, 69.5703])
Video std: tensor([41.4943, 49.9189, 53.7432])
adaptive mean: tensor([20.0841, 21.6957, 21.1346])
adaptive std: tensor([12.6079, 15.1307, 16.2748])
